In [1]:
import numpy as np
import random
import scipy.linalg
import gym
import os
import sys
sys.path.append("../utility")
sys.path.append("../gym_env")
sys.path.append("../")
from utility.Utility import DerivativeLiftFunc, data_collecter

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
ENV = ["DampingPendulum","Pendulum-v1","CartPole-v1","MountainCarContinuous-v0"]

In [3]:
def evalKoopmanD(env_index,samples=20000):
    env_name = ENV[env_index]
    print(env_name)
    Data_collecter = data_collecter(env_name)
    Nstate = Data_collecter.Nstates
    udim = Data_collecter.udim
    LiftFunc = DerivativeLiftFunc(env_name,Nstate,udim)
    NKoopman = LiftFunc.NKoopman
    test_Samples = 5000
    Steps = 30
    random.seed(1)
    np.random.seed(1)
    def A_and_G_o(s_1, s_2, u): # Uses measurements s(t_k) & s(t_{k+1}) to calculate A and G
        A = np.dot(LiftFunc.Psi_su(s_2, u).reshape(-1,1), LiftFunc.Psi_su(s_1, u).reshape(1,-1))
        # print(A.shape)
        G = np.dot(LiftFunc.Psi_su(s_1, u).reshape(-1,1), LiftFunc.Psi_su(s_1, u).reshape(1,-1))
        return A, G

    np.random.seed(1)
    env = Data_collecter.env
    env.reset()
    Samples = samples*15
    A = np.zeros((NKoopman+1, NKoopman+1))
    G = np.zeros((NKoopman+1, NKoopman+1))

    Ps0_list = np.empty((Samples,NKoopman+1))
    Psi_list = np.empty((Samples,NKoopman+1))

    for i in range (Samples):

        # Sample states
        s0 = Data_collecter.random_state()
        u10 = np.random.uniform(Data_collecter.umin, Data_collecter.umax)

        # Simulate system forward
        env.reset_state(s0)
        sn = env.step(u10)
        sn = sn[0]
        # sn = odeint(single_pendulum, s0, [0, ts], args=(u10,))
        # sn = sn[-1,:]   

        # Evaluate basis functions at t = 0 and t = ts
        Ps0_list[i,:] = LiftFunc.Psi_su(s0, u10).reshape(-1)
        Psi_list[i,:] = LiftFunc.Psi_su(sn, u10).reshape(-1)

        [Atemp, Gtemp] = A_and_G_o(s0,sn,u10)
        A = A+Atemp
        G = G+Gtemp

    Kd = np.dot(A,scipy.linalg.pinv(G)) # more accurate than numpy
    # print("The identified Koopman operator is \n", Kd)
    eig = np.linalg.eigvals(Kd)
    print("max eig val:{}".format(max(eig)))

    ## Measure maximum local (across one time step) errors in Ψ(s_{k+1}) - Kd*Ψ(s_k)
    local_errors = np.empty([Samples, NKoopman+1])
    for i in range(Samples):
        local_errors[i,:] = abs(Psi_list[i,:]- np.dot(Kd,Ps0_list[i,:]))
    max_local_errors = np.amax(local_errors, axis = 0)
    print('Max local errors in theta: {}'.format(max_local_errors[:Nstate]))
    np.save("../Prediction_Results/Samples/"+"Kd_"+env_name+"_KoopmanDerivativesample{}".format(sample)+".npy",Kd)  
    times = 4 
    max_loss_all = np.zeros((times,Steps))
    mean_loss_all = np.zeros((times,Steps))
    for t in range(times):
        test_data_path = "../Data/{}{}.npy".format(env_name,t)
        if os.path.exists(test_data_path):
            test_data = np.load("../Data/{}{}.npy".format(env_name,t))
        else:
            test_data = Data_collecter.collect_koopman_data(test_Samples,Steps)
            np.save("../Data/{}{}.npy".format(env_name,i),test_data)
        X_current = test_data[0,:,:]
        X_current_lift = np.zeros((test_Samples,NKoopman+udim))
        maxloss_list = []
        meanloss_list = []
        for i in range(test_Samples):
            X_current_lift[i] = LiftFunc.Psi_su(X_current[i,udim:],X_current[i,:udim])
        for i in range(Steps):
            X_current_lift = np.dot(X_current_lift,Kd.T)
            X_current_lift[:,NKoopman:] = test_data[i+1,:,:udim]
            Err = X_current_lift[:,:Nstate] - test_data[i+1,:,udim:]
            maxloss = np.mean(np.max(abs(Err),axis=0))
            meanloss = np.mean(np.mean(abs(Err),axis=0))
            maxloss_list.append(maxloss)
            meanloss_list.append(meanloss)
        max_loss_all[t] = np.array(maxloss_list).reshape(-1)
        mean_loss_all[t] = np.array(meanloss_list).reshape(-1)
    max_mean = np.mean(max_loss_all,axis=0)
    max_std = np.std(max_loss_all,axis=0)
    mean_mean =  np.mean(mean_loss_all,axis=0)
    mean_std =  np.std(mean_loss_all,axis=0)  
    np.save("../Prediction_Results/Samples/"+env_name+"_KoopmanDerivativesamples{}".format(samples)+".npy",np.array([max_mean,max_std,mean_mean,mean_std]))
    return max_mean,max_std,mean_mean,mean_std 

In [4]:
for sample in [200, 1000, 5000, 20000, 50000]:
     evalKoopmanD(0,sample)

DampingPendulum


max eig val:(0.9999999999999996+0j)
Max local errors in theta: [0.00163031 0.1622848 ]
DampingPendulum
max eig val:(1.0000000000000004+0j)
Max local errors in theta: [0.00161491 0.16054463]
DampingPendulum
max eig val:(1.0000000000000002+0j)
Max local errors in theta: [0.00158888 0.15841206]
DampingPendulum
max eig val:(1.0000000000000004+0j)
Max local errors in theta: [0.0015913  0.15865589]
DampingPendulum
max eig val:(0.999999999999999+0j)
Max local errors in theta: [0.00159068 0.15851301]


In [5]:
# for sample in [200]:
#      evalKoopmanD(1,sample)

In [6]:
# for sample in [200]:
#      evalKoopmanD(2,sample)

In [7]:
# for sample in [200]:
#      evalKoopmanD(3,sample)